In [1]:
import importlib as imp
import sys
import numpy as np
sys.path = list(set(sys.path + ['../../']))

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"
import tensorflow as tf

In [3]:
from sklearn import datasets
dataset_iris = datasets.load_iris(return_X_y=True)

In [4]:
def generate_sample_dataset_and_feature_dict(dataset_iris):
    """
        Make dataset from iris, assuming a user likes class 0 but dislikes class 1 and class 2
    """
    npd = np.zeros(len(dataset_iris[0]), dtype=[
        ('user_id', np.int32), ('item_id', np.int32), ('label', np.int32)])
    fea_dict = dict()
    for idx in range(len(dataset_iris[0])):
        if dataset_iris[1][idx] == 0:
            npd[idx] = (0, idx, 1)
        else:
            npd[idx] = (0, idx, -1)
        fea_dict[idx] = dataset_iris[0][idx]
    return npd, fea_dict

data_all, fea_dict = generate_sample_dataset_and_feature_dict(dataset_iris)

In [5]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(data_all)
print(len(data_train), len(data_test))

112 38


In [6]:
# create dataset instance from structured ndarray data
import recsys.dataset as dataset
dataset = imp.reload(dataset)
train_dataset = dataset.Dataset(data_train, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Train')
test_dataset = dataset.Dataset(data_test, total_users=1, total_items=len(fea_dict), implicit_negative=False, name='Test')

In [7]:
# create featurizer for mapping: (user, item) -> vector
import recsys.featurizers.plain_featurizer as plain_featurizer
plain_featurizer = imp.reload(plain_featurizer)
class Featurizer(plain_featurizer.PlainFeaturizer):
    """
        Custom featurizer
    """
    def __init__(self, fea_dict):
        self.fea_dict = fea_dict

    def feature_dim(self):
        """
            Dim of X
        """
        return 4
    
    def featurize(self, user_id, item_id):
        if user_id != 0:
            raise "error"
        return self.fea_dict[item_id]

featurizer_iris = Featurizer(fea_dict)

In [8]:
# create data sampler for training and testing
import recsys.samplers.ranknet_sampler as ranknet_sampler
ranknet_sampler = imp.reload(ranknet_sampler)
train_sampler = ranknet_sampler.create_training_sampler(
    dataset=train_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, batch_size=10, num_process=1, seed=100)
test_sampler = ranknet_sampler.create_evaluation_sampler(
    dataset=test_dataset, featurizer=featurizer_iris, max_pos_neg_per_user=5, seed=10)

In [9]:
# create evaluation metric
import recsys.metrics.auc as auc
auc = imp.reload(auc)
auc_evaluator = auc.AUC()

In [10]:
# create model
import recsys.recommenders.factorization_machine_recommender as factorization_machine_recommender
factorization_machine_recommender = imp.reload(factorization_machine_recommender)
model = factorization_machine_recommender.FactorizationMachineRecommender(feature_dim=featurizer_iris.feature_dim(),
    factor_dim=5, init_model_dir=None, save_model_dir='FMRec', l2_reg=None, train=True, serve=True)

In [11]:
# create model trainer with model
import recsys.model_trainer as model_trainer
model_trainer = imp.reload(model_trainer)
trainer = model_trainer.ModelTrainer(model=model)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [12]:
import datetime
print(datetime.datetime.now())

2019-08-02 11:52:38.836323


In [13]:
# train
trainer.train(total_iter=100, 
                    eval_iter=1,
                    save_iter=5,
                    train_sampler=train_sampler,
                    eval_samplers=[test_sampler], 
                    evaluators=[auc_evaluator])

[Training starts, total_iter: 100, eval_iter: 1, save_iter: 5]
[iter 1] loss: 0.011301
..(dataset: Test) evaluation
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 2] loss: 0.015675
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 3] loss: 0.017888
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 4] loss: 0.030305
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 5] Model saved.
[iter 5] loss: 0.027336
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 6] loss: 0.005882
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0


INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 47] loss: 0.004059
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 48] loss: 0.004812
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 49] loss: 0.005239
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
Instructions for updating:
Use standard file APIs to delete files with this prefix.
[iter 50] Model saved.
[iter 50] loss: 0.003810
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 51] loss: 0.001140
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 52] loss: 0.001092
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt


INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 93] loss: 0.001890
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 94] loss: 0.000475
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 95] Model saved.
[iter 95] loss: 0.000359
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 96] loss: 0.000368
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 97] loss: 0.000248
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 98] loss: 0.000223
..(dataset: Test) evaluation
INFO:tensorflow:Restoring parameters from FMRec/model.ckpt
..(dataset: Test) AUC 1.0
[iter 99] loss: 0.000076
..(dataset: Test) evaluation
INFO

In [14]:
import datetime
print(datetime.datetime.now())

2019-08-02 11:52:53.737632


In [15]:
# Export Protobuf model for online serving
!rm -rf ./pbModel
model.export(export_model_dir="pbModel", as_text=False)

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: pbModel/saved_model.pb


In [16]:
# inspect internal op tensor value of training
b = train_sampler.next_batch()
print("data:", b)
print("dy, dy_tilde:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_outputs()))
print("loss:", model.train_inspect_ports(batch_data=b, ports=model.traingraph.get_losses()))

data: [([5.4, 3.9, 1.7, 0.4], [6.3, 3.4, 5.6, 2.4], 1)
 ([6.3, 3.4, 5.6, 2.4], [5.4, 3.9, 1.7, 0.4], 0)
 ([5.4, 3.9, 1.7, 0.4], [6. , 2.2, 4. , 1. ], 1)
 ([6. , 2.2, 4. , 1. ], [5.4, 3.9, 1.7, 0.4], 0)
 ([5.4, 3.9, 1.7, 0.4], [5.7, 2.8, 4.1, 1.3], 1)
 ([5.7, 2.8, 4.1, 1.3], [5.4, 3.9, 1.7, 0.4], 0)
 ([5.4, 3.9, 1.7, 0.4], [6.9, 3.1, 5.1, 2.3], 1)
 ([6.9, 3.1, 5.1, 2.3], [5.4, 3.9, 1.7, 0.4], 0)
 ([5.4, 3.9, 1.7, 0.4], [5.6, 3. , 4.5, 1.5], 1)
 ([5.6, 3. , 4.5, 1.5], [5.4, 3.9, 1.7, 0.4], 0)
 ([5. , 3.5, 1.3, 0.3], [6.3, 3.4, 5.6, 2.4], 1)
 ([6.3, 3.4, 5.6, 2.4], [5. , 3.5, 1.3, 0.3], 0)
 ([5. , 3.5, 1.3, 0.3], [6. , 2.2, 4. , 1. ], 1)
 ([6. , 2.2, 4. , 1. ], [5. , 3.5, 1.3, 0.3], 0)
 ([5. , 3.5, 1.3, 0.3], [5.7, 2.8, 4.1, 1.3], 1)
 ([5.7, 2.8, 4.1, 1.3], [5. , 3.5, 1.3, 0.3], 0)
 ([5. , 3.5, 1.3, 0.3], [6.9, 3.1, 5.1, 2.3], 1)
 ([6.9, 3.1, 5.1, 2.3], [5. , 3.5, 1.3, 0.3], 0)
 ([5. , 3.5, 1.3, 0.3], [5.6, 3. , 4.5, 1.5], 1)
 ([5.6, 3. , 4.5, 1.5], [5. , 3.5, 1.3, 0.3], 0)]
dy, dy_tilde:

In [17]:
# inspect internal op tensor value of testing
b = test_sampler.next_batch()
print("data:", b)
print("score:", model.serve_inspect_ports(batch_data=b[1], ports=model.servegraph.get_outputs()))

data: ([1, 1, 1, 1, 1, -1, -1, -1, -1, -1], array([([5.1, 3.5, 1.4, 0.2],), ([4.9, 3. , 1.4, 0.2],),
       ([5.7, 3.8, 1.7, 0.3],), ([4.6, 3.6, 1. , 0.2],),
       ([4.8, 3. , 1.4, 0.1],), ([6.2, 2.9, 4.3, 1.3],),
       ([6.7, 3.1, 5.6, 2.4],), ([5.8, 2.7, 4.1, 1. ],),
       ([6.7, 3.1, 4.7, 1.5],), ([7.4, 2.8, 6.1, 1.9],)],
      dtype=[('x', '<f4', (4,))]))
score: [array([ -3.3898633,  -3.428282 ,  -4.091165 ,  -2.511597 ,  -3.3659272,
        -9.766567 , -12.803475 ,  -9.130553 , -10.815271 , -13.9883375],
      dtype=float32)]


In [18]:
# inspect internal graph weights
def inspect_weights(checkpoint_model_dir):
    reader = tf.train.NewCheckpointReader(checkpoint_model_dir)
    variables = reader.get_variable_to_shape_map()
    for var, shape in variables.items():
        trained_weight = reader.get_tensor(var)
        print("VarName: {}".format(var))
        print("VarShape: {}".format(shape))
        print("VarValue: {}".format(trained_weight))
inspect_weights("./FMRec/model.ckpt")

VarName: optimizergraph/interactiongraph/LinearComponent/bias__0/Adam_1
VarShape: [1]
VarValue: [0.]
VarName: optimizergraph/interactiongraph/LinearComponent/bias__0/Adam
VarShape: [1]
VarValue: [0.]
VarName: optimizergraph/interactiongraph/LinearComponent/FC__0/Adam
VarShape: [4, 1]
VarValue: [[ 0.00114381]
 [-0.00103294]
 [ 0.00368713]
 [ 0.0014334 ]]
VarName: optimizergraph/interactiongraph/InteractiveComponent/interaction_factors/Adam
VarShape: [4, 5]
VarValue: [[ 0.00292579 -0.00294396 -0.00298136 -0.00301317  0.0029555 ]
 [ 0.00120348 -0.00120777 -0.00122022 -0.00123404  0.00121097]
 [-0.0021197   0.00206895  0.00203726  0.00208101 -0.00205752]
 [-0.00041266  0.00039877  0.00037926  0.00039202 -0.00039182]]
VarName: optimizergraph/beta2_power
VarShape: []
VarValue: 0.903888463973999
VarName: optimizergraph/beta1_power
VarShape: []
VarValue: 2.3905195121187717e-05
VarName: interactiongraph/LinearComponent/FC__0
VarShape: [4, 1]
VarValue: [[-2.8521913e-01]
 [ 1.2129910e-03]
 [-1.50